In [6]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from jim_decoding_functions import *


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [7]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [8]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [9]:
task='GlobalLocal'

conditions = stimulus_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskg', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'g'}
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskl with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskl', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'l'}
  Loading condition: Stimulus/BigLetters/SmallLetters/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetters/Taskg', 'bigLetter': 's', 'smallLetter': 's', 'ta

In [10]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/BigLetters/SmallLetterh/Taskg',
 'Stimulus/BigLetters/SmallLetterh/Taskl',
 'Stimulus/BigLetters/SmallLetters/Taskg',
 'Stimulus/BigLetters/SmallLetters/Taskl',
 'Stimulus/BigLetterh/SmallLetterh/Taskg',
 'Stimulus/BigLetterh/SmallLetterh/Taskl',
 'Stimulus/BigLetterh/SmallLetters/Taskg',
 'Stimulus/BigLetterh/SmallLetters/Taskl']

get significant channels

In [11]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [12]:
# Assuming you have a list of subjects
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"


In [13]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

let's manually make the occ filtered sig electrodes

In [14]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [15]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [16]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [17]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [18]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [19]:
# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

subjects:  ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
num channels:  46
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskg: 59
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskl: 54
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskg: 53
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskl: 34
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetterh/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetterh/SmallLetterh/Taskg: 51
sub:  D00

unstack the dat dict so that it is trials, channels x timepoints. This removes ALL data currently, because each channel is nan for at least some trials, because it had to be filled in so that the subjects could all have the same number of channels. 9/6.

#### Unstacked Data Dictionary (unstacked_dat) Structure
- **roi1**
  - **data**: 2D data array for this ROI (trials, channels x timepoints)
  - **channel_names**: List of significant channels for this ROI, as keys in a dictionary, with their channel indices as values
  - **channel_idx**: 1D array where each entry is an integer index corresponding to a channel_name, each entry corresponds to a column
  - **cond_names**: Dictionary where keys are condition names and values are integer indices (unchanged from dat)
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name, each entry corresponds to a trial
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 2D data array for this ROI (trials, channels x timepoints)
  - **channel_names**: List of significant channels for this ROI, as keys in a dictionary, with their channel indices as values
  - **channel_idx**: 1D array where each entry is an integer index corresponding to a channel_name, each entry corresponds to a column
  - **cond_names**: Dictionary where keys are condition names and values are integer indices (unchanged from dat)
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name, each entry corresponds to a trial
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 2D data array for this ROI (trials, channels x timepoints)
  - **channel_names**: List of significant channels for this ROI, as keys in a dictionary, with their channel indices as values
  - **channel_idx**: 1D array where each entry is an integer index corresponding to a channel_name, each entry corresponds to a column
  - **cond_names**: Dictionary where keys are condition names and values are integer indices (unchanged from dat)
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name, each entry corresponds to a trial
  - **times**: 1D array of times, corresponding to each sample


        




In [20]:
import numpy as np

def unstack_data_without_nans(data, cond_idx):
    """
    Unstacks the data from its original shape (trials, channels, timepoints)
    to (trials * channels, timepoints), and removes any rows that contain NaNs.
    
    Args:
        data (np.ndarray): A 3D numpy array with shape (trials, channels, timepoints).
        cond_idx (np.ndarray): A 1D numpy array containing the condition indices for each trial.
    
    Returns:
        tuple: A tuple containing:
            - clean_unstacked_data (np.ndarray): Unstacked data with NaN rows removed.
            - clean_cond_indices (np.ndarray): Condition indices corresponding to clean trials.
            - clean_channel_indices (np.ndarray): Channel indices corresponding to clean trials.
    
    Prints:
        - The original shape of the data.
        - The shape of the unstacked data.
        - The number of good (non-NaN) trials and their corresponding indices.
    """
    # Reshape data from (trials, channels, timepoints) to (trials, channels*timepoints)
    unstacked_data = data.reshape(data.shape[0], -1)  # Shape: (trials, channels*timepoints)

    # Create the trial indices based on the original structure
    unstacked_trial_indices = np.arange(data.shape[0])
    # Create the channel indices based on the original structure
    unstacked_channel_indices = np.repeat(np.arange(data.shape[1]), data.shape[2])

    # Identify which columns (channels*timepoints) have NaNs in any trial
    nan_mask_per_column = np.isnan(unstacked_data).any(axis=0)

    # Filter out the columns (channels * timepoints) with NaNs
    clean_unstacked_data = unstacked_data[:, ~nan_mask_per_column]
    
    # Filter the channel indices based on the columns that are kept
    clean_channel_indices = unstacked_channel_indices[~nan_mask_per_column]

    # The trial indices and condition indices remain the same
    clean_trial_indices = np.arange(data.shape[0])
    clean_cond_indices = cond_idx

    # Print the results
    print(f"Original data shape: {data.shape}")
    print(f"Unstacked data shape: {unstacked_data.shape}")
    print(f"Cleaned unstacked data shape: {clean_unstacked_data.shape}")
    print(f"Number of good (unstacked) columns: {clean_unstacked_data.shape[1]}")
    print(f"Number of good (unstacked) trials: {clean_unstacked_data.shape[0]}")

    return clean_unstacked_data, clean_cond_indices, clean_channel_indices


def unstack_dat_for_each_roi(dat, rois):
    """
    Processes the data for each region of interest (ROI) by unstacking the data,
    converting the channel names to a dictionary, and filtering out rows with NaNs.
    
    Args:
        dat (dict): A dictionary containing the data for each ROI. Each ROI's data includes:
            - 'channel_names' (list): List of channel names.
            - 'cond_names' (dict): Dictionary of condition names with their corresponding indices.
            - 'data' (np.ndarray): 3D numpy array of shape (trials, channels, timepoints).
            - 'cond_idx' (np.ndarray): Array of condition indices for each trial.
        rois (list): List of regions of interest (ROIs) to process.
    
    Returns:
        dict: A dictionary where each key is an ROI, and the value is another dictionary containing:
            - 'cond_names' (dict): Condition names with their indices.
            - 'channel_names' (dict): Channel names mapped to their corresponding indices.
            - 'data' (np.ndarray): Unstacked and cleaned data with no NaNs.
            - 'cond_indices' (np.ndarray): Condition indices for the cleaned data.
            - 'channel_indices' (np.ndarray): Channel indices for the cleaned data.
    """
    unstacked_dat = {}

    for roi in rois:
        unstacked_dat[roi] = {}
        
        # Store the condition names dictionary
        unstacked_dat[roi]['cond_names'] = dat[roi]['cond_names']
        
        # Convert the list of channel names to a dictionary with indices
        channel_names = dat[roi]['channel_names']
        channel_name_dict = {name: idx for idx, name in enumerate(channel_names)}
        
        # Store the dictionary of channel names
        unstacked_dat[roi]['channel_names'] = channel_name_dict
        
        # Unstack the data and filter out NaNs
        unstacked_dat[roi]['data'], unstacked_dat[roi]['cond_idx'], unstacked_dat[roi]['channel_idx'] = unstack_data_without_nans(
            dat[roi]['data'], dat[roi]['cond_idx']
        )

    return unstacked_dat

# Example usage:
unstacked_dat = unstack_dat_for_each_roi(dat, rois)


Original data shape: (3429, 46, 641)
Unstacked data shape: (3429, 29486)
Cleaned unstacked data shape: (3429, 0)
Number of good (unstacked) columns: 0
Number of good (unstacked) trials: 3429
Original data shape: (1926, 12, 641)
Unstacked data shape: (1926, 7692)
Cleaned unstacked data shape: (1926, 0)
Number of good (unstacked) columns: 0
Number of good (unstacked) trials: 1926
Original data shape: (3011, 41, 641)
Unstacked data shape: (3011, 26281)
Cleaned unstacked data shape: (3011, 0)
Number of good (unstacked) columns: 0
Number of good (unstacked) trials: 3011
Original data shape: (2290, 23, 641)
Unstacked data shape: (2290, 14743)
Cleaned unstacked data shape: (2290, 0)
Number of good (unstacked) columns: 0
Number of good (unstacked) trials: 2290
Original data shape: (1079, 10, 641)
Unstacked data shape: (1079, 6410)
Cleaned unstacked data shape: (1079, 0)
Number of good (unstacked) columns: 0
Number of good (unstacked) trials: 1079


make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


In [50]:
import numpy as np

def put_data_in_labeled_array_per_roi_subject(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
    """
    Organize the MNE data into separate LabeledArray for each ROI and subject, with NaN padding for trial dimension.
    Concatenates subject data along the channels axis.
    
    Parameters:
    - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
    - condition_names: List of condition names.
    - rois: List of region of interest (ROI) names.
    - subjects: List of subjects.
    - sig_electrodes_per_subject_roi: Dictionary that maps ROIs to subjects and their corresponding electrodes.
    
    Returns:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI. 
                          Each LabeledArray concatenates the subject data along the channels dimension.
                          Dimensions of LabeledArray: [Conditions, Electrodes (concatenated), Trials, Timepoints]
    """
    roi_labeled_arrays = {}

    # Loop through each ROI
    for roi in rois:
        roi_labeled_array = None

        # Find the maximum number of trials for each condition across all subjects
        max_trials_per_condition = {condition: 0 for condition in condition_names}

        # First pass to find the max number of trials per condition
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
            if not sig_electrodes:
                continue
            for condition_name in condition_names:
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                epochs_data = epochs.get_data(copy=True)  # Shape: (trials, channels, timepoints)
                max_trials_per_condition[condition_name] = max(max_trials_per_condition[condition_name], epochs_data.shape[0])

        # Second pass to gather data for each subject and condition
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
            if not sig_electrodes:
                continue

            subject_nested_dict = {}

            # Get channel names for this subject's ROI
            sub_channel_names = [f"{sub}-{sig_electrode}" for sig_electrode in sig_electrodes]

            # Loop through each condition
            for condition_name in condition_names:
                # Extract the epoch data for the current condition and subject, pick only the electrodes of interest
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                epochs_data = epochs.get_data(copy=True)  # Shape: (trials, channels, timepoints)
                # Get the number of trials and the target number of trials
                n_trials, n_channels, n_timepoints = epochs_data.shape
                max_trials = max_trials_per_condition[condition_name]

                # Pad with NaNs if necessary
                if n_trials < max_trials:
                    # Create a padded array with NaNs
                    padded_data = np.full((max_trials, n_channels, n_timepoints), np.nan)
                    padded_data[:n_trials, :, :] = epochs_data  # Copy existing data into the padded array
                else:
                    padded_data = epochs_data

                subject_nested_dict[condition_name] = padded_data

            # Create a LabeledArray for the subject
            subject_labeled_array = LabeledArray.from_dict(subject_nested_dict)
            subject_labeled_array.labels[2] = sub_channel_names  # Add channel labels for this subject
            
            if roi_labeled_array is None:
                roi_labeled_array = subject_labeled_array
            else:
                roi_labeled_array = roi_labeled_array.concatenate(subject_labeled_array, axis=2) # concatenate along channels axis

        # Add the concatenated LabeledArray to the ROI dictionary
        roi_labeled_arrays[roi] = roi_labeled_array

    return roi_labeled_arrays


In [88]:
# Example call to the function
roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

In [108]:
roi_labeled_arrays['lpfc']['Stimulus/BigLetters/SmallLetterh/Taskg'][58]

array([[-0.2708306 , -0.25662053, -0.25457552, ...,  0.41792989,
         0.38908193,  0.34015727],
       [ 0.35916242,  0.1691516 , -0.03048072, ...,  1.26573491,
         1.47234237,  1.66987419],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])
labels(['D0057-RAI6', 'D0057-RPI14', 'D0059-LMMF9', 'D0059-LMMF11', 'D0059-LMMF12', 'D0059-LPSF16', 'D0063-LMSF5', 'D0063-LPSF12', 'D0063-RAI4', 'D0063-RAI6', 'D0063-RMMF13', 'D0063-RMMF14', 'D0063-RMMF10', 'D0065-RASF14', 'D0065-RASF15', 'D0065-RI7', 'D0071-RFO14', 'D0071-RIA4', 'D0071-RIP6', 'D0071-RIA5', 'D0071-RIA11', 'D0071-RIA12', 'D0071-RIA14', 'D0071-RIA16', 'D0090-RIA6', 'D00

convert the time labels from sample indices to timepoints (this breaks the indexing for some reason, fix this later with aaron 9/7)

In [53]:
# def get_event_times_as_list_of_strings(subjects_mne_objects):
#     """
#     Extract the event times from the first subject's MNE object and convert them to a list of strings.

#     Parameters:
#     - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}

#     Returns:
#     - times_as_strings: List of event times converted to strings, corresponding to sample time points.
#     """
#     first_subject = next(iter(subjects_mne_objects.keys()))
#     first_condition = next(iter(subjects_mne_objects[first_subject].keys()))
#     times = subjects_mne_objects[first_subject][first_condition]['HG_ev1_rescaled'].copy().times
#     times_as_strings = [str(t) for t in times]

#     return times_as_strings


# def convert_roi_labeled_arrays_sample_labels_to_times_as_strings(subjects_mne_objects, roi_labeled_arrays):
#     """
#     Replace the sample labels (axis 3) of each ROI's LabeledArray in roi_labeled_arrays 
#     with event times (as strings).

#     Parameters:
#     - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
#     - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.

#     Returns:
#     - roi_labeled_arrays: Updated dictionary with timepoints as strings for the 3rd axis labels of each LabeledArray.
#     """
#     times_as_strings = get_event_times_as_list_of_strings(subjects_mne_objects)
#     rois = list(roi_labeled_arrays.keys())

#     for roi in rois:
#         roi_labeled_arrays[roi].labels[3] = times_as_strings

#     return roi_labeled_arrays

# roi_labeled_arrays_time_labels = convert_roi_labeled_arrays_sample_labels_to_times_as_strings(subjects_mne_objects, roi_labeled_arrays)

playing around with smashing conditions and trials 9/7

In [102]:
combined_conditions_trials_array.shape

(512, 46, 641)

In [76]:
labeled_array = roi_labeled_arrays['lpfc']

combined_conditions_trials_array = labeled_array.combine((0,1))

# Drop trials where all values are NaN
combined_conditions_trials_array_clean = combined_conditions_trials_array.dropna()

# Combine the channels and timepoints axes (axis 1 and axis 2) into a single axis
channel_time_combined_array = combined_conditions_trials_array_clean.combine((1, 2))


In [98]:
roi_labeled_arrays['lpfc'].labels

[['Stimulus/BigLetters/SmallLetterh/Taskg', 'Stimulus/BigLetters/SmallLetterh/Taskl', 'Stimulus/BigLetters/SmallLetters/Taskg', 'Stimulus/BigLetters/SmallLetters/Taskl', 'Stimulus/BigLetterh/SmallLetterh/Taskg', 'Stimulus/BigLetterh/SmallLetterh/Taskl', 'Stimulus/BigLetterh/SmallLetters/Taskg', 'Stimulus/BigLetterh/SmallLetters/Taskl'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63'],
 ['D0057-RAI6', 'D0057-RPI14', 'D0059-LMMF9', 'D0059-LMMF11', 'D0059-LMMF12', 'D0059-LPSF16', 'D0063-LMSF5', 'D0063-LPSF12', 'D0063-RAI4', 'D0063-RAI6', 'D0063-RMMF13', 'D0063-RMMF14', 'D0063-RMMF10', 'D0065-RASF14', 'D0065-RASF15', 'D0065-RI7', 'D0071-RFO14', 'D0071-RIA4', 'D0071-RI

In [105]:
condition_data = roi_labeled_arrays['lpfc']['Stimulus/BigLetters/SmallLetterh/Taskg']
print(condition_data.shape)
# Drop trials where all values are NaN. This is not working! 9/7
condition_data_clean = condition_data.dropna()
print(condition_data_clean.shape)

condition_data_clean[58]

(64, 46, 641)
(59, 46, 641)


array([[-0.2708306 , -0.25662053, -0.25457552, ...,  0.41792989,
         0.38908193,  0.34015727],
       [ 0.35916242,  0.1691516 , -0.03048072, ...,  1.26573491,
         1.47234237,  1.66987419],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])
labels(['D0057-RAI6', 'D0057-RPI14', 'D0059-LMMF9', 'D0059-LMMF11', 'D0059-LMMF12', 'D0059-LPSF16', 'D0063-LMSF5', 'D0063-LPSF12', 'D0063-RAI4', 'D0063-RAI6', 'D0063-RMMF13', 'D0063-RMMF14', 'D0063-RMMF10', 'D0065-RASF14', 'D0065-RASF15', 'D0065-RI7', 'D0071-RFO14', 'D0071-RIA4', 'D0071-RIP6', 'D0071-RIA5', 'D0071-RIA11', 'D0071-RIA12', 'D0071-RIA14', 'D0071-RIA16', 'D0090-RIA6', 'D00

In [89]:
def reshape_labeled_array_for_decoding(labeled_array):
    """
    Reshape a single LabeledArray, removing NaN trials and reshaping to be trials by channelsxtimepoints.

    Parameters:
    - labeled_array: A LabeledArray where the 0th axis labels are conditions, 1st axis labels are trials, 
                     2nd axis labels are channels, and 3rd axis labels are timepoints.

    Returns:
    - reshaped_data: Dictionary where keys are condition names and values are reshaped arrays (trials x channels*timepoints).
    """
    reshaped_labeled_array = {}
    
    # Extract the condition labels (which are in labeled_array.labels[0])
    condition_names = labeled_array.labels[0]

    # Loop over each condition
    for condition_name in condition_names:
        # Extract the data for the current condition
        condition_data = labeled_array[condition_name]

        # Drop trials where all values are NaN. This is not working! 9/7
        condition_data_clean = condition_data.dropna()

        # Combine the channels and timepoints axes (axis 1 and axis 2) into a single axis
        channel_time_combined_array = condition_data_clean.combine((1, 2))
        
        # Store the processed data for this condition
        reshaped_labeled_array[condition_name] = channel_time_combined_array

    return reshaped_labeled_array

def process_all_rois_for_decoding(roi_labeled_arrays):
    """
    Loop through all ROIs and apply reshaping function to each LabeledArray.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.

    Returns:
    - all_processed_data: Dictionary where keys are ROIs and values are dictionaries of reshaped data for each condition.
    """
    reshaped_roi_labeled_arrays_for_decoding = {}

    # Loop through each ROI in the dictionary
    for roi, labeled_array in roi_labeled_arrays.items():
        # Apply the reshaping function to the current labeled array
        reshaped_labeled_array = reshape_labeled_array_for_decoding(labeled_array)

        # Store the reshaped data for this ROI
        reshaped_roi_labeled_arrays_for_decoding[roi] = reshaped_labeled_array

    return reshaped_roi_labeled_arrays_for_decoding

reshaped_roi_labeled_arrays_for_decoding = process_all_rois_for_decoding(roi_labeled_arrays)

In [95]:
reshaped_roi_labeled_arrays_for_decoding['lpfc']['Stimulus/BigLetterh/SmallLetterh/Taskg'][58]

array([nan, nan, nan, ..., nan, nan, nan])
labels(['D0057-RAI6-0', 'D0057-RAI6-1', 'D0057-RAI6-2', 'D0057-RAI6-3', 'D0057-RAI6-4', 'D0057-RAI6-5', 'D0057-RAI6-6', 'D0057-RAI6-7', 'D0057-RAI6-8', 'D0057-RAI6-9', 'D0057-RAI6-10', 'D0057-RAI6-11', 'D0057-RAI6-12', 'D0057-RAI6-13', 'D0057-RAI6-14', 'D0057-RAI6-15', 'D0057-RAI6-16', 'D0057-RAI6-17', 'D0057-RAI6-18', 'D0057-RAI6-19', 'D0057-RAI6-20', 'D0057-RAI6-21', 'D0057-RAI6-22', 'D0057-RAI6-23', 'D0057-RAI6-24', 'D0057-RAI6-25', 'D0057-RAI6-26', 'D0057-RAI6-27', 'D0057-RAI6-28', 'D0057-RAI6-29', 'D0057-RAI6-30', 'D0057-RAI6-31', 'D0057-RAI6-32', 'D0057-RAI6-33', 'D0057-RAI6-34', 'D0057-RAI6-35', 'D0057-RAI6-36', 'D0057-RAI6-37', 'D0057-RAI6-38', 'D0057-RAI6-39', 'D0057-RAI6-40', 'D0057-RAI6-41', 'D0057-RAI6-42', 'D0057-RAI6-43', 'D0057-RAI6-44', 'D0057-RAI6-45', 'D0057-RAI6-46', 'D0057-RAI6-47', 'D0057-RAI6-48', 'D0057-RAI6-49', 'D0057-RAI6-50', 'D0057-RAI6-51', 'D0057-RAI6-52', 'D0057-RAI6-53', 'D0057-RAI6-54', 'D0057-RAI6-55', 'D0057-

attempt to do time resolved decoding 8/21

In [73]:
def create_time_bins(time_points, bin_size=0.1):
    """
    Create time bins from the given time points array.

    Parameters:
    - time_points: 1D numpy array of time points.
    - bin_size: The size of each time bin in seconds (default is 0.1s, which is 100 ms).

    Returns:
    - bins: A list of numpy arrays, where each array contains the time points within that bin.
    """
    bins = []
    start_time = np.min(time_points)
    end_time = np.max(time_points)
    current_time = start_time
    
    while current_time < end_time:
        next_time = current_time + bin_size
        if next_time > end_time:
            next_time = end_time
        bins.append(time_points[(time_points >= current_time) & (time_points < next_time)])
        current_time = next_time

    # Ensure that the last bin includes the endpoint
    if time_points[-1] not in bins[-1]:
        bins[-1] = np.append(bins[-1], time_points[-1])

    return bins

In [74]:
# Assuming temporal_datasets is a dictionary
first_roi = next(iter(dat.keys()))
time_points = dat[first_roi]['times']

# Create time bins
bins = create_time_bins(time_points, bin_size=0.1)
bins

[array([-1.        , -0.99609375, -0.9921875 , -0.98828125, -0.984375  ,
        -0.98046875, -0.9765625 , -0.97265625, -0.96875   , -0.96484375,
        -0.9609375 , -0.95703125, -0.953125  , -0.94921875, -0.9453125 ,
        -0.94140625, -0.9375    , -0.93359375, -0.9296875 , -0.92578125,
        -0.921875  , -0.91796875, -0.9140625 , -0.91015625, -0.90625   ,
        -0.90234375]),
 array([-0.8984375 , -0.89453125, -0.890625  , -0.88671875, -0.8828125 ,
        -0.87890625, -0.875     , -0.87109375, -0.8671875 , -0.86328125,
        -0.859375  , -0.85546875, -0.8515625 , -0.84765625, -0.84375   ,
        -0.83984375, -0.8359375 , -0.83203125, -0.828125  , -0.82421875,
        -0.8203125 , -0.81640625, -0.8125    , -0.80859375, -0.8046875 ,
        -0.80078125]),
 array([-0.796875  , -0.79296875, -0.7890625 , -0.78515625, -0.78125   ,
        -0.77734375, -0.7734375 , -0.76953125, -0.765625  , -0.76171875,
        -0.7578125 , -0.75390625, -0.75      , -0.74609375, -0.7421875 ,
     

try to implement ieeg example decoding using reshaped roi labeled arrays 9/7

In [69]:
reshaped_roi_labeled_arrays_for_decoding['lpfc'].keys()

dict_keys(['Stimulus/BigLetters/SmallLetterh/Taskg', 'Stimulus/BigLetters/SmallLetterh/Taskl', 'Stimulus/BigLetters/SmallLetters/Taskg', 'Stimulus/BigLetters/SmallLetters/Taskl', 'Stimulus/BigLetterh/SmallLetterh/Taskg', 'Stimulus/BigLetterh/SmallLetterh/Taskl', 'Stimulus/BigLetterh/SmallLetters/Taskg', 'Stimulus/BigLetterh/SmallLetters/Taskl'])

In [ ]:
roi = 'occ'

# Create a Decoder and run cross-validation
cats = {group: i for i, group in enumerate(reshaped_roi_labeled_arrays_for_decoding[roi].keys())}

decoder = Decoder(cats, 0.80, oversample=True, n_splits=5, n_repeats=5)

# Use your unstacked data for the decoder
cm = decoder.cv_cm(unstacked_dat[roi]['data'], labels, normalize='true')
cm = np.mean(cm, axis=0)

# Plot the Confusion Matrix
fig, ax = plt.subplots()
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cats.keys())
disp.plot(ax=ax)
plt.savefig('decoding_test.png')

try to implement the ieeg example decoding using our unstacked dat 9/4

In [ ]:
# def relabel_conditions_for_decoding(cond_idx: np.ndarray, grouped_conditions: dict) -> np.ndarray:
#     """
#     Relabels the condition indices based on the grouped conditions.

#     Args:
#         cond_idx (np.ndarray): The original condition indices for each trial.
#         grouped_conditions (dict): A dictionary where keys are group names (e.g., "BigLetters", "BigLetterh")
#                                    and values are lists of condition indices for each group.
    
#     Returns:
#         np.ndarray: A new array of labels, where trials are relabeled according to the groups.
#                     The shape of this array matches the shape of cond_idx.
#     """
#     # Create an empty array to store the new labels
#     new_labels = np.full_like(cond_idx, -1)  # Initialize with -1 as a placeholder

#     # Assign label 0 for conditions in "BigLetters"
#     for idx in grouped_conditions['BigLetters']:
#         new_labels[cond_idx == idx] = 0

#     # Assign label 1 for conditions in "BigLetterh"
#     for idx in grouped_conditions['BigLetterh']:
#         new_labels[cond_idx == idx] = 1

#     return new_labels

# # Example usage of grouping based on condition indices:
# if conditions == stimulus_conditions:
#     grouped_conditions = {
#         "BigLetters": [0, 1, 2, 3],  # Group 0
#         "BigLetterh": [4, 5, 6, 7]   # Group 1
#     }

# # Assume the unstacked data has been processed and we are working with one ROI
# roi = 'occ'  # Example region of interest
# cond_idx = unstacked_dat[roi]['cond_idx']  # Get the condition indices for the current ROI

# # Relabel the cond_idx using the chosen grouped conditions
# labels = relabel_conditions_for_decoding(cond_idx, grouped_conditions)

# # Now 'labels' contains the new labels (0 for BigLetters and 1 for BigLetterh)
# print("Original cond_idx:", cond_idx)
# print("New labels:", labels)

Original cond_idx: [0 0 0 ... 7 7 7]
New labels: [0 0 0 ... 1 1 1]


In [ ]:
# roi = 'occ'

# # Create a Decoder and run cross-validation
# cats = {group: i for i, group in enumerate(grouped_conditions.keys())}
# decoder = Decoder(cats, 0.80, oversample=True, n_splits=5, n_repeats=5)

# # Use your unstacked data for the decoder
# cm = decoder.cv_cm(unstacked_dat[roi]['data'], labels, normalize='true')
# cm = np.mean(cm, axis=0)

# # Plot the Confusion Matrix
# fig, ax = plt.subplots()
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cats.keys())
# disp.plot(ax=ax)
# plt.savefig('decoding_test.png')

KeyboardInterrupt: 

now actually run the decoding (my implementation, old as of 9/4)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC  # Example: Support Vector Classifier

def decode_conditions_time_binning(dat, conditions, classifier='logistic_regression', cv_folds=5, bin_size=0.1, classify_by='bigLetter', rois=None):
    decoding_results = {}

    # Choose the classifier
    if classifier == 'logistic_regression':
        clf = make_pipeline(LogisticRegression(max_iter=1000))
    elif classifier == 'hist_gradient_boosting':
        clf = make_pipeline(HistGradientBoostingClassifier())
    elif classifier == 'svm':
        clf = make_pipeline(SVC(kernel='linear'))  # Example: Linear SVM
    else:
        raise ValueError(f"Unsupported classifier: {classifier}")

    # Determine which ROIs to run the decoding on
    if rois is None:
        rois = dat.keys()  # Default to all ROIs in dat
    else:
        rois = [roi for roi in rois if roi in dat]

    # Extract time points from the first ROI in the `dat`
    first_roi = next(iter(dat.keys()))
    time_points = dat[first_roi].labels[-1]  # Assuming the last label is time_points

    # Create time bins
    bins = create_time_bins(time_points, bin_size=bin_size)

    # Create a map from condition names to the chosen attribute (e.g., bigLetter, smallLetter, or task)
    condition_map = {name: conditions[name][classify_by] for name in conditions}

    # Get unique labels for classification
    unique_labels = list(set(condition_map.values()))
    label_to_index = {label: i for i, label in enumerate(unique_labels)}

    # Loop over the selected ROIs in the data dictionary
    for roi in rois:
        roi_data = dat[roi]
        print(f'Decoding conditions for ROI: {roi}')

        # Extract data and original condition labels
        data = roi_data  # Since roi_data is already a LabeledArray
        cond_idx = data.labels[1]  # Assuming the second label dimension is condition indices
        sub_idx = data.labels[0]  # Assuming the first label dimension is subject indices

        # Create group labels based on the chosen attribute
        group_idx = np.array([label_to_index[condition_map[cond_name]] for cond_name in conditions.keys()])[cond_idx]

        # Calculate window indices based on the bins
        time_window_results = []
        num_trials, num_channels, _ = data.shape

        for time_bin in bins:
            bin_indices = np.isin(time_points, time_bin)
            data_window = data[:, :, bin_indices]  # Select data within the time bin

            # Identify and remove all-NaN channels
            nan_channels = np.all(np.isnan(data_window), axis=(0, 2))  # Find channels that are all NaNs
            data_window_valid_channels = data_window[:, ~nan_channels, :]  # Keep only non-NaN channels

            # Flatten the data for decoding
            num_valid_channels = data_window_valid_channels.shape[1]
            data_window_flat = data_window_valid_channels.reshape(num_trials, num_valid_channels * len(time_bin))

            # Perform cross-validated decoding between the two groups
            cv = StratifiedKFold(n_splits=cv_folds)
            scores = cross_val_score(clf, data_window_flat, group_idx, cv=cv)

            # Store the decoding accuracy for this time bin
            mean_accuracy = np.mean(scores)
            std_accuracy = np.std(scores)
            time_window_results.append({
                'time_window': (time_bin[0], time_bin[-1]),
                'mean_accuracy': mean_accuracy,
                'std_accuracy': std_accuracy
            })

            print(f"Time bin {time_bin[0]}s to {time_bin[-1]}s: Mean accuracy = {mean_accuracy:.4f}")

        # Store results for each ROI
        decoding_results[roi] = time_window_results

    return decoding_results

In [ ]:
# # To classify based on the 'bigLetter' attribute for all ROIs
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.2, classify_by='bigLetter')

# To classify based on the 'bigLetter' attribute for specific ROIs
# specific_rois = ['occ_best_filtered']
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.3, classify_by='bigLetter', rois=specific_rois)

occ_decoding_results = decode_conditions_time_binning(
    dat, 
    stimulus_conditions, 
    classifier='hist_gradient_boosting', 
    cv_folds=5, 
    bin_size=0.3, 
    classify_by='bigLetter', 
    rois=['occ']
)

# # To classify based on the 'smallLetter' attribute for specific ROIs
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.1, classify_by='smallLetter', rois=specific_rois)

# # To classify based on the 'task' attribute for all ROIs
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.1, classify_by='task')


condition map:  {'Stimulus/BigLetters/SmallLetterh/Taskg': 's', 'Stimulus/BigLetters/SmallLetterh/Taskl': 's', 'Stimulus/BigLetters/SmallLetters/Taskg': 's', 'Stimulus/BigLetters/SmallLetters/Taskl': 's', 'Stimulus/BigLetterh/SmallLetterh/Taskg': 'h', 'Stimulus/BigLetterh/SmallLetterh/Taskl': 'h', 'Stimulus/BigLetterh/SmallLetters/Taskg': 'h', 'Stimulus/BigLetterh/SmallLetters/Taskl': 'h'}
unique labels:  ['s', 'h']
label to index:  {'s': 0, 'h': 1}
Decoding conditions for ROI: occ


KeyboardInterrupt: 

define colors for plotting (not used yet as of 8/21)

In [ ]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}